## Crop Yield Prediction 

Aim: Predict crop yields for american crops for this year, based on current and historical images. 

### Requirements:
 <br>
 1. Map layers from the USDA that mark farmed land. <br>
 2. High frequency optical color data of the farmed land. <br>
 3. weather data that can be readily incorporated.  <br>

In [1]:
import ee

In [2]:
%%capture captured_output
ee.Authenticate()

Enter verification code: 4/1AfJohXmRW5VQmbpdLTktol28TmqLWza3Er54LU9awnrgFd8TuLeSLvqkptk


In [3]:
ee.Initialize()

### 1. Map layers from the USDA 

In [4]:
import datetime
import geemap.core as geemap

In [135]:
%matplotlib notebook

# Initialize a map object.
m = geemap.Map()
center_coordinates = [-100, 40]
USDA_cropland = ee.ImageCollection("USDA/NASS/CDL").filterDate('2020-01-01', '2020-12-31').first()
m.add_layer(USDA_cropland, None, 'USDA CDL Layer')
m.centerObject(ee.Geometry.Point(center_coordinates), 4)

In [136]:
m

Map(center=[40, -100], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

What we want to extract from this CDL layer is the cropland for corn. from the reference page on (https://developers.google.com/earth-engine/datasets/catalog/USDA_NASS_CDL#bands) we can see that the code for corn is 1 and the associated color is #ffd300 (yellow). Lets visualize this. 

In [137]:
cropland_only = geemap.Map()
center_coordinates = [-100, 40]

# USDA CDL With layers 'cropland' detailing the types of crops, 'cultivated' (0 = uncultivated)
# confidence (0-100)

corn_farmland = USDA_cropland.select("cropland")
confidence_classification = USDA_cropland.select('confidence')


cropland_only.addLayer(corn_farmland,{},'cropland')
cropland_only.centerObject(ee.Geometry.Point(center_coordinates), 4)
cropland_only

Map(center=[40, -100], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

Lets also extract just the corn farmland from this data and create a mask. 

In [19]:

corn_only_map = geemap.Map()

center_coordinates = [-100, 40]

mask  = corn_farmland.eq(1)

## 1.1 Producing training data 

To create a training dataset, we would have to first select a patch of land (or construct a polygon of coordinates) over which we have both CDL and Landsat coverage. Ordered by volume, the largest produces of corn are Iowa, Illinois and Nebraska. Lets select Iowa for our analysis.

In [23]:
all_states = ee.FeatureCollection('TIGER/2018/States')

# state code for nebraska is '31', Iowa is '19' and illinois is '17'
iowa_geometry = all_states.filter(ee.Filter.eq('STATEFP', '19')).geometry()

# Add the Nebraska polygon to the map
corn_only_map.addLayer(iowa_geometry, {"color": "lightblue"},'Nebraska')


Now we'll get LandSat images covering Iowa. 

In [24]:
iowa_bbox = iowa_geometry.bounds()

In [25]:
#enlarged_iowa_bbox = iowa_geometry.bounds().buffer(500000)
coords = iowa_bbox.getInfo()['coordinates'][0]

# Create a bounding box using the coordinates
iowa_bbox_geometry = ee.Geometry.Polygon(coords)

# Calculate the centroid coordinates of Iowa
centroid_coords = iowa_geometry.centroid().coordinates().getInfo()

# Create a bounding box around the centroid to ensure it covers the entire state
enlarged_iowa_bbox = ee.Geometry.Point(centroid_coords)

image_from_landsat = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')\
                        .filterBounds(enlarged_iowa_bbox)\
                        .filterDate('2020-05-01', '2020-08-31')\
                        .sort('CLOUD_COVER').first()\
              

### Landsat Image Bands 


In [74]:
img_geo = image_from_landsat.geometry()

In [26]:
image_only = geemap.Map()
center_coordinates = [-100, 40]

vis_params = {
    'bands': ['B4', 'B3', 'B2'],  # True color bands
    'min': 0,
    'max': 3000
}
image_only.addLayer(image_from_landsat,vis_params,'LandSat Image')
image_only.addLayer(iowa_geometry, {"color": "lightblue"},'Nebraska')
image_only.addLayer(cornRegions.clip(image_from_landsat.geometry()), {"color": "red", "opacity": 1})
image_only.centerObject(ee.Geometry.Point(centroid_coords), 7)

#image_only.addLayer(cornRegions, {"color": "red", "opacity": 1})

In [104]:
corn_farmland = USDA_cropland.select("cropland")
mask = corn_farmland.eq(1)
farm_vector = mask.reduceToVectors(
    geometry=img_geo,
    scale=100,# Choose an appropriate scale
    geometryType='centroid')

In [105]:
corn_points = farm_vector.geometry()

In [106]:
map_gee = geemap.Map()
map_gee.addLayer(cornRegions.clip(image_from_landsat.geometry()), {"color": "red", "opacity": 1})
map_gee.addLayer(corn_points,{'color':'red'},'cpoints')
map_gee

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

In [112]:
import numpy as np 

fifty_random_idxs = np.random.randint(0, len(corn_points.getInfo()['coordinates']),size=50)

In [127]:
coords = corn_points.getInfo()['coordinates']

random_coords = [ coords[i] for i in fifty_random_idxs ] 

points_random = [ee.Geometry.Point(i) for i in random_coords ]

In [130]:
points_collection = ee.FeatureCollection(points_random)

In [133]:
map_gee = geemap.Map()
map_gee.addLayer(cornRegions.clip(image_from_landsat.geometry()), {"color": "red", "opacity": 1})
map_gee.addLayer(points_collection,{'color':'red'},'random points')

In [134]:
map_gee

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

Average NDVI over 50 points 